<a href="https://colab.research.google.com/github/shreekar2005/Cheminformatics_DC/blob/main/Cheminformatics_DC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Tox21 Dense features

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
print("hi")
#load data with urls
url1='https://raw.githubusercontent.com/shreekar2005/Cheminformatics_DC/refs/heads/main/DATASET_sorted/TRAIN_dense/tox21_dense_train.csv'
url2='https://raw.githubusercontent.com/shreekar2005/Cheminformatics_DC/refs/heads/main/DATASET_sorted/TRAIN_dense/tox21_labels_train.csv'
url3='https://raw.githubusercontent.com/shreekar2005/Cheminformatics_DC/refs/heads/main/DATASET_sorted/TEST_dense/tox21_dense_test.csv'
url4='https://raw.githubusercontent.com/shreekar2005/Cheminformatics_DC/refs/heads/main/DATASET_sorted/TEST_dense/tox21_labels_test.csv'
train_features = pd.read_csv(url1)
train_labels = pd.read_csv(url2)
test_features = pd.read_csv(url3)
test_labels = pd.read_csv(url4)

# List of target labels (Ensure these are correct)
target_labels = ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD',
                 'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']

# Select only dense numerical features (Exclude non-numeric columns)
dense_features = train_features.select_dtypes(include=['int64', 'float64']).columns.tolist() # name of dense features

X_train = train_features[dense_features].values  # Extract only numerical features
# y_train = train_labels[target_labels].values     # Extract corresponding target labels

X_test = test_features[dense_features].values
# y_test = test_labels[target_labels].values

# Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert NaN values to 0 in labels
train_labels[target_labels] = train_labels[target_labels].fillna(0)
test_labels[target_labels] = test_labels[target_labels].fillna(0)

# Ensure all values are either 0 or 1
train_labels[target_labels] = train_labels[target_labels].clip(0, 1)
test_labels[target_labels] = test_labels[target_labels].clip(0, 1)
'''
what does .clip(0,1) do ?
If an element is:
Less than 0, it is set to 0.
Greater than 1, it is set to 1.
Between 0 and 1, it remains unchanged
'''

# Convert labels to numpy arrays and PyTorch tensors
y_train = train_labels[target_labels].values
y_test = test_labels[target_labels].values


# Convert to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)  # Multi-label classification

X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Move tensors to GPU (if available)
X_train_torch, y_train_torch = X_train_torch.to(device), y_train_torch.to(device)
X_test_torch, y_test_torch = X_test_torch.to(device), y_test_torch.to(device)


cuda


#NN

building model

In [ ]:
class neural_network(nn.Module):
    def __init__(self, input_dim):
        super(neural_network, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 12)  # 12 neurons for 12 labels
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # Sigmoid for multi-label classification

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))  # Sigmoid for multi-label probabilities
        return x

training model


In [ ]:
model1 = neural_network(X_train.shape[1]).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multi-label classification
optimizer = optim.Adam(model1.parameters(), lr=0.001)

for epoch in range(50):  # Train for 50 epochs
    model1.train()
    optimizer.zero_grad()
    outputs = model1(X_train_torch)
    loss = criterion(outputs, y_train_torch)  # Loss computed for all 12 labels
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.6854
Epoch 10, Loss: 0.3613
Epoch 20, Loss: 0.2439
Epoch 30, Loss: 0.2030
Epoch 40, Loss: 0.1804


validating model

In [ ]:
model1.eval()
with torch.no_grad():
    # X_test_torch, y_test_torch = X_test_torch.to(device), y_test_torch.to(device)
    test_outputs = model1(X_test_torch)
    test_loss = criterion(test_outputs, y_test_torch)
    print(f"Test Loss: {test_loss.item():.4f}")

# Convert outputs to numpy for accuracy calculation
test_outputs_np = test_outputs.cpu().numpy()
y_test_np = y_test_torch.cpu().numpy()

# ===== Step 8: Compute Accuracy for Each Label =====
threshold = 0.5  # Convert probabilities to binary (0 or 1)
test_preds = (test_outputs_np >= threshold).astype(int)

print("\nAccuracy for each label:")
for i, label in enumerate(target_labels):
    acc = accuracy_score(y_test_np[:, i], test_preds[:, i])
    print(f"{label}: {acc:.4f}")

Test Loss: 0.2082

Accuracy for each label:
NR.AhR: 0.8872
NR.AR: 0.9815
NR.AR.LBD: 0.9876
NR.Aromatase: 0.9397
NR.ER: 0.9212
NR.ER.LBD: 0.9691
NR.PPAR.gamma: 0.9521
SR.ARE: 0.8563
SR.ATAD5: 0.9413
SR.HSE: 0.9660
SR.MMP: 0.9057
SR.p53: 0.9366


#KNN


In [ ]:
class KNNClassifier:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2, axis=1))  # Vectorized distance

    def predict(self, X_test):
        X_test = np.array(X_test)
        predictions = []

        for x in X_test:
            # Compute distances from test point to all training points
            distances = self._euclidean_distance(self.X_train, x)
            # Get indices of k nearest neighbors
            k_indices = np.argsort(distances)[:self.k]
            # Get corresponding labels
            k_labels = self.y_train[k_indices]

            # Majority voting for multi-label classification (mean method)
            y_pred = (np.mean(k_labels, axis=0) >= 0.5).astype(int)
            predictions.append(y_pred)

        return np.array(predictions)

In [ ]:
# Train custom KNN model
knn = KNNClassifier(k=5)
knn.fit(X_train, y_train)

# Predict on test data
y_pred = knn.predict(X_test)

# Compute accuracy per label
label_accuracies = {}
for i, col in enumerate(y_train.columns):
    correct = np.sum(y_train[col].values == y_pred[:, i])
    label_accuracies[col] = correct / len(y_train)

# Print accuracy for each label
for label, acc in label_accuracies.items():
    print(f"Accuracy for {label}: {acc:.4f}")

# trying on tox21 sparse features



In [ ]:
import pandas as pd
from scipy.io import mmread

# Updated URL (If confirmed it is the wrong file, get the new URL)
# If the intended format is `csv`, reading it directly with pandas should resolve the issue
# example: url = 'https://github.com/shreekar2005/Cheminformatics_DC/raw/main/DATASET_sorted/TRAIN_sparse/tox21_sparse_train.csv'
#           df = pd.read_csv(url)
url = 'https://raw.githubusercontent.com/shreekar2005/Cheminformatics_DC/refs/heads/main/DATASET_sorted/TRAIN_sparse/tox21_sparse_train.mtx'

try:
    # Attempt to read as Matrix Market file
    Matrix = mmread(url)
    B = Matrix.todense()
    df = pd.DataFrame(B, range(1, B.shape[0] + 1), range(1, B.shape[1] + 1))
    print(df)

except ValueError as e:
    if "Not a Matrix Market file" in str(e):
        # If it's not a Matrix Market file, try reading as a CSV with pandas
        print("File is not in Matrix Market format. Trying to read as CSV.")
        df = pd.read_csv(url)
        print(df)
    else:
        # If another error occurred, raise it
        raise e

File is not in Matrix Market format. Trying to read as CSV.
        %%MatrixMarket matrix coordinate integer general
0                                   12060 272776 3913072
1                                                  1 1 4
2                                                  4 1 2
3                                                  7 1 1
4                                                 20 1 2
...                                                  ...
3913068                                   12058 270072 1
3913069                                   12058 270073 1
3913070                                   12060 270074 1
3913071                                   12060 270075 1
3913072                                   12060 270076 1

[3913073 rows x 1 columns]
